Word Vectors
==========
Ref: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors

Goal: Using distributed word vectors created by the Word2Vec algorithm. Word2Vec (Google, 2013) is a neural network implementation that learns distributed representations for words. It does not need labels. 

Using word2vec in Python
--------------
We will use **gensim** package. We also need to install **cython** otherwise it will take days to run instead of minutes.

Preparing to Train a Model
------------------


In [1]:
import pandas as pd

# Read data from files
train=pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t",quoting=3)
test=pd.read_csv("testData.tsv", header=0, delimiter="\t",quoting=3)
unlabeled_train=pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" \
%(train["review"].size, test["review"].size, unlabeled_train["review"].size)

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



To train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. It is also better not to remove numbers.

In [6]:
# Import various modules for string cleaning 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.
    # 1. Remove HTML
    review_text=BeautifulSoup(review,"html").get_text()
    # 2. Remove non-letters, keep numbers
    review_text=re.sub("[^a-zA-Z0-9]"," ",review_text)
    # 3. Convert words in lower case and split them
    words=review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops=set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
    # 5. return a list of words
    return(words)

Word2Vec expects single sentences, each one as a list of words. It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's **punkt** tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [7]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()  #I've done this before

# Load the punkt tokenizer
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

In [41]:
# Define a function to split a review into parsed sentences
def review_to_sentences(review,tokenizer,remove_stopwords=False):
    # Function to split a review into parsed sentences. 
    # Returns a list of sentences, where each sentence is a list of words.
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences=tokenizer.tokenize(review.strip())
    # 2. Loop over each sentence
    sentences=[]
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence)>0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
        # Return the list of sentences (each sentence is a list of words, so this returns a list of lists)
    return sentences
    

In [44]:
sentences = []

print "Parsing sentences from training set"
for review in train["review"]:
    review=review.decode("utf8")
    sentences+=review_to_sentences(review, tokenizer)
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    review=review.decode("utf8")
    sentences+=review_to_sentences(review, tokenizer)


Parsing sentences from training set
Parsing sentences from unlabeled set


/Users/anw4/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/anw4/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % mar

In [45]:
# Check how many sentences we have in total- should be around 850,000+
print len(sentences)
print sentences[0]
print sentences[1]

795538
[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


A minor detail to note is the difference between the "+=" and "append" when it comes to Python lists. In many applications the two are interchangeable, but here they are not. If you are appending a list of lists to another list of lists, "append" will only append the first list; you need to use "+=" in order to join all of the lists at once.
Training and Saving Your Model
-------------------
- **Architecture**: skip-gram (default, slightly slower but produced better results) or continuous bag of words. 
- **Training algorithm**: Hierarchical softmax (default) or negative sampling.
- **Downsampling of frequent words**: ranges from .00001 to .001. We use 0.001, which seems to improve the accuracy of the final model.
- **Word vector dimensionality**: ranges from tens to thousands. We use 300. More features result in longer runtimes and often result in better models.
- **Context / window size**: 10 seems to work well for hierarchical softmax. 
- **Minimum word count**: ranges from 10 to 100. We set to 40 to avoid attaching too much importance to individual movie titles. This results in an overall vocabulary size of ~15,000 words.  

In [46]:
# Import the built-in logging module and configure it so that Word2Vec creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s',level=logging.INFO)

# Set values efor various parameters
num_features = 300 # word vector dimensionality
min_word_count = 40 
num_workers= 4 # number of threads to run in parallel
context= 10  # context window size
downsampling= 1e-3 # downsample setting for frequent words

# Initialize and train the model (take some time)
from gensim.models import word2vec
print "Training model..."
model=word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count,\
                       window=context, sample=downsampling)
#If you don't plan to train the model any further, calling init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and save the model for later use.
# You can load it later using Word2Vec.load()
model_name="300features_40minwords_10context"
model.save(model_name)


Training model...


Exploring the Model Results
------------------


In [51]:
# Function "doesnt_match" can deduce which word in a set is most dissimilar from the others:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [55]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [56]:
# Function "most_similar" can get insight into the model's word clusters:
model.most_similar("man")

[(u'woman', 0.6284092664718628),
 (u'lady', 0.5979578495025635),
 (u'lad', 0.5781458020210266),
 (u'men', 0.5217606425285339),
 (u'guy', 0.5173251032829285),
 (u'soldier', 0.5171582102775574),
 (u'monk', 0.5156159400939941),
 (u'businessman', 0.5037267208099365),
 (u'farmer', 0.5015547871589661),
 (u'millionaire', 0.4996732473373413)]

In [57]:
model.most_similar("queen")

[(u'princess', 0.647876501083374),
 (u'latifah', 0.6203631162643433),
 (u'victoria', 0.6088835000991821),
 (u'maid', 0.607053279876709),
 (u'bride', 0.6010995507240295),
 (u'belle', 0.6008758544921875),
 (u'stepmother', 0.593067467212677),
 (u'goddess', 0.5916730165481567),
 (u'prince', 0.579993486404419),
 (u'dame', 0.5708566904067993)]

In [58]:
model.most_similar("awful")

[(u'terrible', 0.762367844581604),
 (u'horrible', 0.7417336106300354),
 (u'atrocious', 0.7307676076889038),
 (u'dreadful', 0.7018641233444214),
 (u'abysmal', 0.6951020359992981),
 (u'horrendous', 0.6836704015731812),
 (u'horrid', 0.6676329374313354),
 (u'appalling', 0.6660313606262207),
 (u'lousy', 0.6142610311508179),
 (u'amateurish', 0.6114339828491211)]

In [59]:
model.most_similar("great")

[(u'wonderful', 0.7354174256324768),
 (u'terrific', 0.7346562147140503),
 (u'fantastic', 0.7278304696083069),
 (u'superb', 0.6601248979568481),
 (u'fine', 0.6429445743560791),
 (u'excellent', 0.6395628452301025),
 (u'brilliant', 0.6257189512252808),
 (u'good', 0.6159985065460205),
 (u'marvelous', 0.6155490279197693),
 (u'fabulous', 0.5945401191711426)]

We have a reasonably good model for semantic meaning- at least as good as Bag of Words. Next, we are going to use these fancy distributed word vectors for supervised learning.  